---
title: "Data Cleaning"
---

### Import Libraries

In [39]:
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

### Load Dataframe

In [40]:
movies_df = pd.read_csv("../data/raw/movies.csv")

print(len(movies_df))

7333


### Movies With Identical Titles

In [41]:
# Find all duplicated titles
duplicate_titles = movies_df[movies_df.duplicated(subset = "Title", keep = False)]

# Find true duplicates (same title and same year)
true_duplicates = movies_df[movies_df.duplicated(subset = ["Title", "Year"], keep = False)]

# Get non-true duplicates (same title, different year)
mask = duplicate_titles.index.difference(true_duplicates.index)

# Append year to title for non-true duplicates
movies_df.loc[mask, "Title"] = (
    movies_df.loc[mask, "Title"] + " (" + movies_df.loc[mask, "Year"].astype(str) + ")"
)

# Drop true duplicate movies
movies_df = movies_df.drop_duplicates(subset = ["Title", "Year"], keep = "first")

### Convert Data Types

In [42]:
# Convert columns to specified data types
movies_df["Year"] = pd.to_numeric(movies_df["Year"], errors = "coerce").astype("Int64")

movies_df["Runtime"] = movies_df["Runtime"].str.extract(r"(\d+)").astype("Int64")

movies_df["Metascore_Rating"] = pd.to_numeric(movies_df["Metascore_Rating"], errors = "coerce").astype("Int64") / 10
movies_df["Rotten_Tomatoes_Rating"] = movies_df["Rotten_Tomatoes_Rating"].str.rstrip('%').astype("Int64") / 10
movies_df["IMDB_Rating"] = pd.to_numeric(movies_df["IMDB_Rating"], errors = "coerce")
movies_df["TMDB_Rating"] = movies_df["TMDB_Rating"].round(1)

movies_df["Keywords"] = movies_df["Keywords"].str.split(", ")

movies_df["Release_Date"] = pd.to_datetime(movies_df["Release_Date"], errors = "coerce")
movies_df["Release_Month"] = movies_df["Release_Date"].dt.month_name()

movies_df["Awards"] = movies_df["Awards"].astype(str).str.lower()

### Create Additional Columns

In [43]:
# Alter Awards column to only include oscar counts and create new columns based on nominations vs wins
movies_df["Awards"] = movies_df["Awards"].apply(lambda x: x.split("oscar")[0] + "oscar" if "oscar" in str(x) else np.nan)
movies_df["Oscar_Wins"] = movies_df["Awards"].apply(lambda x: int(x.split("won")[1].split("oscar")[0].strip()) if "won" in str(x) else 0)
movies_df["Oscar_Nominations"] = movies_df["Awards"].apply(lambda x: int(x.split("nominated for")[1].split("oscar")[0].strip()) if "nominated" in str(x) else 0)
movies_df = movies_df.drop(columns = ["Awards"])

# Add boolean column for oscar win or not
movies_df["Won_Award"] = movies_df["Oscar_Wins"] != 0

# Return_On_Investment column calculated by dividing movie revenue by budget
movies_df["Return_On_Investment"] = movies_df["Revenue"] / movies_df["Budget"]

# Average_Rating column calculated by obatining the mean of all provided rating scores
movies_df["Average_Rating"] = movies_df[["Metascore_Rating", "IMDB_Rating", "Rotten_Tomatoes_Rating", "TMDB_Rating"]].mean(axis = 1)

# Z-Score normalization for Budget and Revenue columns
movies_df["Budget_Normalized"] = (movies_df["Budget"] - movies_df["Budget"].mean()) / movies_df["Budget"].std()
movies_df["Revenue_Normalized"] = (movies_df["Revenue"] - movies_df["Revenue"].mean()) / movies_df["Revenue"].std()

### Remove Inf and -Inf Values

In [44]:
# Replace infinity values with NaN, then remove NaN
movies_df.replace([np.inf, -np.inf], np.nan, inplace = True)
print(len(movies_df))
# movies_df.dropna(inplace = True)

7332


### Clean Column Types

In [45]:
import ast
# for each row in the Keywords column, convert string representation of list to actual list
movies_df['Keywords'] = movies_df['Keywords'].apply(lambda x: ast.literal_eval(','.join(x)) if isinstance(x, list) else x)

### Rearrange Columns

In [46]:
# Rearrange Columns in movies_df 
movies_df = movies_df[["IMDB_ID", "Title", "Year", "Release_Date", "Release_Month", "Age_Rating", 
                    "Overview", "Keywords", "Genre", "Directors", "Actors", "Producers", "Writers", "Composers", 
                    "Runtime", "Cinematographers", "Franchise_Name", "Production_Companies", "Country",
                    "Language", "Metascore_Rating", "IMDB_Rating", "Rotten_Tomatoes_Rating", "TMDB_Rating", "Average_Rating", 
                    "Won_Award", "Oscar_Wins", "Oscar_Nominations", "Budget", "Budget_Normalized", "Revenue", 
                    "Revenue_Normalized", "Return_On_Investment", "Popularity"]]

### Drop Columns

In [47]:
movies_df = movies_df[movies_df["Revenue"] != 0]

print(len(movies_df))

6940


### Preview Cleaned Dataframe

In [48]:
print(f"Total Movie Count: {len(movies_df)}\n")

print(f"Clean Datatset Shape: {movies_df.shape}\n")

Total Movie Count: 6940

Clean Datatset Shape: (6940, 34)



In [49]:
movies_df.info()
movies_df.describe()

<class 'pandas.core.frame.DataFrame'>
Index: 6940 entries, 0 to 6940
Data columns (total 34 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   IMDB_ID                 6805 non-null   object        
 1   Title                   6940 non-null   object        
 2   Year                    6802 non-null   Int64         
 3   Release_Date            6921 non-null   datetime64[ns]
 4   Release_Month           6921 non-null   object        
 5   Age_Rating              4541 non-null   object        
 6   Overview                6885 non-null   object        
 7   Keywords                6940 non-null   object        
 8   Genre                   6940 non-null   object        
 9   Directors               6940 non-null   object        
 10  Actors                  6940 non-null   object        
 11  Producers               6940 non-null   object        
 12  Writers                 6940 non-null   object       

,Year,Release_Date,Runtime,Metascore_Rating,IMDB_Rating,Rotten_Tomatoes_Rating,TMDB_Rating,Average_Rating,Oscar_Wins,Oscar_Nominations,Budget,Budget_Normalized,Revenue,Revenue_Normalized,Return_On_Investment,Popularity
count,6802.0,6921,6746.0,4953.0,6676.000000,5465.0,6940.000000,6940.0,6940.000000,6940.000000,6.940000e+03,6940.000000,6.940000e+03,6940.000000,5.297000e+03,6940.000000
mean,2003.104822,2003-12-09 09:35:05.071521408,110.551883,5.794327,6.572034,6.211747,6.457594,6.240598,0.162824,0.225072,2.977571e+07,0.033690,1.020252e+08,0.029609,4.913505e+04,4.302702
min,1914.0,1914-04-10 00:00:00,6.0,0.1,1.300000,0.0,0.000000,0.0,0.000000,0.000000,0.000000e+00,-0.625498,1.000000e+00,-0.524204,5.000000e-04,0.000000
25%,1996.0,1996-07-19 00:00:00,95.0,4.5,6.000000,4.1,6.100000,5.3,0.000000,0.000000,1.372500e+04,-0.625194,8.212310e+06,-0.479626,1.324981e+00,1.029475
50%,2008.0,2008-07-25 00:00:00,107.0,5.8,6.600000,6.7,6.600000,6.45,0.000000,0.000000,1.200000e+07,-0.359837,3.809388e+07,-0.317423,2.632831e+00,2.302250
75%,2016.0,2016-11-21 00:00:00,122.0,7.1,7.300000,8.6,7.200000,7.4,0.000000,0.000000,4.000000e+07,0.260041,1.119062e+08,0.083246,5.079769e+00,4.515750
max,2025.0,2025-07-21 00:00:00,960.0,10.0,9.900000,10.0,10.000000,10.0,11.000000,11.000000,4.600000e+08,9.558198,2.923706e+09,15.346272,1.000042e+08,827.910300
std,18.278419,NaN,26.161545,1.766198,1.001617,2.716969,1.357739,1.621579,0.768677,0.811578,4.556687e+07,1.008781,1.878796e+08,1.019849,1.990570e+06,16.896609


In [50]:
movies_df.head()

,IMDB_ID,Title,Year,Release_Date,Release_Month,Age_Rating,Overview,Keywords,Genre,Directors,Actors,Producers,Writers,Composers,Runtime,Cinematographers,Franchise_Name,Production_Companies,Country,Language,Metascore_Rating,IMDB_Rating,Rotten_Tomatoes_Rating,TMDB_Rating,Average_Rating,Won_Award,Oscar_Wins,Oscar_Nominations,Budget,Budget_Normalized,Revenue,Revenue_Normalized,Return_On_Investment,Popularity
0,tt0499549,Avatar,2009,2009-12-15,December,PG-13,"In the 22nd century, a paraplegic Marine is di...","[paraplegic, attachment to nature, culture cla...","['Action', 'Adventure', 'Fantasy', 'Science Fi...",['James Cameron'],"['Sam Worthington', 'Zoe Saldaña', 'Sigourney ...","['James Cameron', 'Jon Landau']",['James Cameron'],['James Horner'],162,['Mauro Fiore'],Avatar Collection,"['Dune Entertainment', 'Lightstorm Entertainme...","United States, United Kingdom","English, Spanish",8.3,7.9,8.1,7.6,7.975,True,3,0,237000000,4.621319,2923706026,15.346272,12.336312,22.0122
1,tt4154796,Avengers: Endgame,2019,2019-04-24,April,PG-13,After the devastating events of Avengers: Infi...,"[superhero, time travel, space travel, time ma...","['Adventure', 'Science Fiction', 'Action']","['Anthony Russo', 'Joe Russo']","['Robert Downey Jr.', 'Chris Evans', 'Mark Ruf...",['Kevin Feige'],"['Stephen McFeely', 'Christopher Markus']",['Alan Silvestri'],181,['Trent Opaloch'],The Avengers Collection,['Marvel Studios'],United States,"English, Japanese, Xhosa, German",7.8,8.4,9.4,8.2,8.45,False,0,1,356000000,7.255797,2799439100,14.671726,7.863593,16.8250
2,tt1630029,Avatar: The Way of Water,2022,2022-12-14,December,PG-13,Set more than a decade after the events of the...,"[dying and death, loss of loved one, alien lif...","['Science Fiction', 'Adventure', 'Action']",['James Cameron'],"['Sam Worthington', 'Zoe Saldaña', 'Sigourney ...","['Jon Landau', 'James Cameron']","['James Cameron', 'Amanda Silver', 'Rick Jaffa']",['Simon Franglen'],192,['Russell Carpenter'],Avatar Collection,"['20th Century Studios', 'Lightstorm Entertain...",United States,English,6.7,7.5,7.6,7.6,7.35,True,1,0,460000000,9.558198,2320250281,12.070591,5.044022,23.3720
3,tt34956443,Ne Zha 2,2025,2025-01-29,January,NR,"Following the Tribulation, although the souls ...","[based on myths,legends or folklore, 3d animat...","['Animation', 'Fantasy', 'Adventure', 'Action']",['Yang Yu'],"['Lu Yanting', 'Joseph', 'Han Mo', 'Chen Hao',...",['Liu Wenzhang'],['Yang Yu'],"['Roc Chen', 'Wan Pin Chu', 'Yang Rui']",143,[],Ne Zha Collection,"['Chengdu Coco Cartoon', 'Beijing Enlight Pict...",China,"Mandarin, Chinese, English, Hindi",6.3,8.1,9.6,8.1,8.025,False,0,0,80000000,1.145579,2213230000,11.489663,27.665375,35.9241
4,tt2488496,Star Wars: The Force Awakens,2015,2015-12-15,December,NaN,Thirty years after defeating the Galactic Empi...,"[android, spacecraft, space opera, requel]","['Adventure', 'Action', 'Science Fiction']",['J.J. Abrams'],"['Harrison Ford', 'Mark Hamill', 'Carrie Fishe...","['Bryan Burk', 'Kathleen Kennedy', 'J.J. Abrams']","['Michael Arndt', 'Lawrence Kasdan', 'J.J. Abr...",['John Williams'],138,['Dan Mindel'],Star Wars Collection,"['Lucasfilm Ltd.', 'Bad Robot']","United States, United Kingdom",English,8.0,7.8,9.3,7.3,8.1,False,0,5,245000000,4.798427,2068223624,10.702538,8.441729,9.8780


### Save Processed Dataframe

In [51]:
movies_df.to_csv("../data/processed/cleaned_movies.csv", index = False)

## Prepare Data for Regression

We will drop columns related to the outcome variable and those that wouldn't exist before the release of a movie. Our target variables will be `Revenue_Normalized` and `Average_Rating`.

We also want to deal with the columns that contains lists of strings, such as `Genre`, `Directors`, `Actors`, etc. These lists are a little tricky to handle, so we will transform them into counts of features. For example, we will count the number of unique genres, directors, actors, etc., and create new columns for these counts. We drop the original columns after creating the new ones and also the `num_actors` since it is redundant with all movies showing the 10 top actors, which was specified in the data colletion process.

In [88]:
import pandas as pd
import ast

# Load the data
df = pd.read_csv("../data/processed/cleaned_movies.csv")

# Drop unwanted columns
cols_to_drop = [
    "IMDB_ID", "Title", "Overview", "Keywords", "Metascore_Rating", "IMDB_Rating",
    "Rotten_Tomatoes_Rating", "TMDB_Rating", "Won_Award", "Oscar_Wins",
    "Oscar_Nominations","Revenue", "Return_On_Investment", "Popularity",
    # "Average_Rating", "Revenue_Normalized"
]

df_clean = df.drop(columns=cols_to_drop)

# Remove rows with missing essential features (e.g., budget)
df_clean = df_clean[df_clean["Budget"].notna()]

# Fill or drop NA for other columns (basic handling)
df_clean = df_clean.dropna(subset=["Runtime", "Age_Rating", "Genre", "Language", "Country"])

# Encode release month as numeric
df_clean["Release_Month"] = pd.Categorical(df_clean["Release_Month"])
df_clean["Release_Month"] = df_clean["Release_Month"].cat.codes

# define columns that contain lists of strings
list_cols = ['Genre', 'Directors', 'Actors', 'Producers', 'Writers',
             'Composers', 'Cinematographers', 'Production_Companies']

for col in list_cols:
    # convert string representation of list to actual list
    df_clean[col] = df_clean[col].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else [])
    # count features 
    df_clean[f'num_{col.lower()}'] = df_clean[col].apply(len)

# drop those columns that have lists, also drop num_actors
df_clean = df_clean.drop(columns=list_cols + ["num_actors"])

# Save cleaned version
df_clean.to_csv("../data/processed/cleaned_movies_reg.csv", index=False)